# Face Mask Detection



Here I am going to detect whether a person is wearing mask or not. I am focussing on only two classes that are 'face_with_mask' and face_no_mask'.

### Source Page
+ [Kaggle Source](https://www.kaggle.com/ayushimishra2809/face-mask-detection)


### Setup

1. Create a new [Anaconda](https://www.anaconda.com/products/individual) environment with Python 3.6 (not 3.7 or 3.8), keras, and tensorflow:
``` sh
> conda create -n <your_env_name> python=3.6 anaconda tensorflow keras
```
2. Install other missing packages:
``` sh
> conda install -c anaconda xlrd xlwt scikit-learn seaborn
> conda install mtcnn ipyparallel
> pip install python-opencv
```
3. That's all for now!

## Importing Libraries

In [1]:
import numpy as np 
import pandas as pd 
import os
import urllib
import matplotlib.pyplot as plt
import cv2                                 # opencv-python installed with 'pip'
import matplotlib.patches as patches
import tensorflow as tf                    # TensorFlow installed with 'pip'
import keras
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential        # keras is only compatible with python 3.6 or lower
import sklearn
from sklearn.preprocessing import LabelEncoder
from mtcnn.mtcnn import MTCNN

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


Traceback (most recent call last):
  File "/Users/brtonnies/anaconda3/envs/AI/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-8126c71bfe97>", line 8, in <module>
    import tensorflow as tf                    # TensorFlow installed with 'pip'
  File "/Users/brtonnies/anaconda3/envs/AI/lib/python3.6/site-packages/tensorflow/__init__.py", line 98, in <module>
    from tensorflow_core import *
  File "/Users/brtonnies/anaconda3/envs/AI/lib/python3.6/site-packages/tensorflow_core/__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 947, in _find_and_load_unlocked
  File "/Users/brtonnies/anaconda3/envs/AI/lib/python3.6/site-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "

TypeError: object of type 'NoneType' has no len()

In [ ]:
# pip install mtcnn

In [ ]:
# from mtcnn.mtcnn import MTCNN

# Loading datasets

+ [Correctly Masked Faces](https://esigelec-my.sharepoint.com/:f:/g/personal/cabani_esigelec_fr/Ev3GdnQSyzxPjyzU5ElHqagBlkRCaKnnCI85iX-d1L4OHA?e=G7uaYV)

+ [Incorrectly Masked Faces](https://esigelec-my.sharepoint.com/:f:/g/personal/cabani_esigelec_fr/EirjS8ew7-5LnO8I56Uk63wBKebwSlukFBFBaO8N25wn3g?e=Ho1jHG)

+ [Face Mask Detection Dataset (Kaggle)](https://www.kaggle.com/wobotintelligence/face-mask-detection-dataset)

In [ ]:
# images=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/images")
# annotations=os.path.join("/kaggle/input/face-mask-detection-dataset/Medical mask/Medical mask/Medical Mask/annotations")
# train=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/train.csv"))
# submission=pd.read_csv(os.path.join("/kaggle/input/face-mask-detection-dataset/submission.csv"))
notebook_dir = !pwd
notebook_dir = notebook_dir[0]
root_dir = os.path.abspath(os.path.join(notebook_dir, os.pardir))
data_dir = os.path.join(root_dir, 'data')
images_dir = os.path.join(data_dir, 'images')
annotations_dir = os.path.join(data_dir, 'annotations')


train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
submission = pd.read_csv(os.path.join(data_dir, 'submission.csv'))

In [ ]:
print(len(train))
train

In [ ]:
print(len(submission))
submission.head()

In [ ]:
NUM_IMAGES = len(os.listdir(images_dir))

## We have 6024 images.  There's an extra file in the directory we'll ignore

In [ ]:
image_formats = ['jpg', 'jpeg', 'png']
a = os.listdir(images_dir)
a = [img for img in a if img.split(".")[-1] in image_formats]
b = os.listdir(annotations_dir)
a.sort()
b.sort()

In [ ]:
print(len(b),len(a))

## The Original Source Used a Supervised Learning Model

In [ ]:
train_images=a[1698:]
test_images=a[:1698]


Just to prove there's at least one image

In [ ]:
img=plt.imread(os.path.join(images_dir, test_images[0]))
plt.imshow(img)
plt.show()

In [ ]:
img=plt.imread(os.path.join(images_dir, train_images[1]))
plt.imshow(img)
plt.show()

In [ ]:
options=['face_with_mask','face_no_mask']
train= train[train['classname'].isin(options)]
train.sort_values('name',axis=0,inplace=True)

In [ ]:
bbox=[]
for i in range(len(train)):
    arr=[]
    for j in train.iloc[i][["x1",'x2','y1','y2']]:
        arr.append(j)
    bbox.append(arr)
train["bbox"]=bbox  

def get_boxes(id):
    boxes=[]
    for i in train[train["name"]==str(id)]["bbox"]:
        boxes.append(i)
    return boxes

print(get_boxes(train_images[3]))
image=train_images[3]

img=plt.imread(os.path.join(images_dir,image))

fig,ax = plt.subplots(1)
ax.imshow(img)
boxes=get_boxes(image)
for box in boxes:
    rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=2,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
image=train_images[5]

img=plt.imread(os.path.join(images_dir,image))

fig,ax = plt.subplots(1)
ax.imshow(img)
boxes=get_boxes(image)
for box in boxes:
    rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],linewidth=2,edgecolor='r',facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
plt.bar(['face_with_mask','face_no_mask'],train.classname.value_counts())

# Creating training data

In [ ]:
img_size=50
data=[]
path='../data/images/'
def create_data():
    for i in range(len(train)):
        arr=[]
        for j in train.iloc[i]:
               arr.append(j)
        img_array = cv2.imread(os.path.join(images_dir, arr[0]), cv2.IMREAD_GRAYSCALE)
        crop_image = img_array[arr[2]:arr[4], arr[1]:arr[3]]
        new_img_array=cv2.resize(crop_image, (img_size, img_size))
        data.append([new_img_array, arr[5]])
        
        pct = i/len(train) * 100
        if pct % 5 == 0:
            print("Creating Training Data -- {}%".format(pct))
            
create_data()

len(train)

In [ ]:
data[0][0]
plt.imshow(data[0][0])

In [ ]:
len(data)

In [ ]:
x=[]
y=[]
for features, labels in data:
    x.append(features)
    y.append(labels)

# print(y)
lbl=LabelEncoder()
y=lbl.fit_transform(y)
y.shape

In [ ]:
x=np.array(x).reshape(-1,50,50,1)
x=tf.keras.utils.normalize(x,axis=1)
from keras.utils import to_categorical
y = to_categorical(y)
y

# Model Fitting

In [ ]:
from keras.layers import LSTM
model=Sequential()
model.add(Conv2D(100,(3,3), input_shape=x.shape[1:], activation='relu', strides=2))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))


In [ ]:
# if no previously compiled model, then compile a new one

# does a previously compiled model exist?
if os.path.exists("../models/example_model"):
    model = keras.models.load_model("../models/example_model")
else:
    #  WARNING: THIS TAKES TIME -- USE THE COMPILED MODEL IF POSSIBLE
    opt = keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-5)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x, y, epochs=30, batch_size=5)
    model.save("../models/example_model")

In [ ]:
def get_face_bounds(img):
    detector = MTCNN()
    image = plt.imread(os.path.join(images_dir, img))
    faces = detector.detect_faces(image)
    print(faces)
    bounds = []
    for (x, y, w, h) in [face['box'] for  face in faces]:
        bounding_box = cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 5)
        plt.imshow(bounding_box)
        bounds.append(bounding_box)

    return bounds

In [ ]:
image = test_images[0]
bounding_boxes =  get_face_bounds(image)

In [ ]:
image = test_images[60]
bounding_boxes += get_face_bounds(image)

In [ ]:
image = test_images[3]
bounding_boxes += get_face_bounds(image)

In [ ]:
detector=MTCNN()
test_df=[]
for image in test_images:
    img=plt.imread(os.path.join(images_dir, image))
    faces=detector.detect_faces(img)
    
    test=[]
    for face in faces:
        bounding_box=face['box']
        test.append([image,bounding_box])
    test_df.append(test)
    
test=[]
for i in test_df:
    if len(i)>0:
        if len(i)==1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)

In [ ]:
sub = []
rest_image = []
for i in test:
    sub.append(i[0])
for image in test_images:
    if image not in sub:
        rest_image.append(image) 


detector=MTCNN()
test_df_ = []
for image in rest_image:
    img = cv2.imread(os.path.join(images_dir,image))
    faces = detector.detect_faces(img)
    test_ = []
    for face in faces:
        bounding_box = face['box']
        test_.append([image, bounding_box])
    test_df_.append(test_) 
for i in test_df_:
    if len(i) > 0:
        if len(i) == 1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)      

In [ ]:
sub = []
rest_image = []
for i in test:
    sub.append(i[0])
for image in test_images:
    if image not in sub:
        rest_image.append(image) 


detector=MTCNN()
test_df_ = []
for image in rest_image:
    img = cv2.imread(os.path.join(images_dir,image))
    faces = detector.detect_faces(img)
    test_ = []
    for face in faces:
        bounding_box = face['box']
        test_.append([image, bounding_box])
    test_df_.append(test_) 
for i in test_df_:
    if len(i) > 0:
        if len(i) == 1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)      

In [ ]:
sub = []
rest_image = []
for i in test:
    sub.append(i[0])
for image in test_images:
    if image not in sub:
        rest_image.append(image) 


detector=MTCNN()
test_df_ = []
for image in rest_image:
    img = cv2.imread(os.path.join(images_dir,image))
    faces = detector.detect_faces(img)
    test_ = []
    for face in faces:
        bounding_box = face['box']
        test_.append([image, bounding_box])
    test_df_.append(test_) 
for i in test_df_:
    if len(i) > 0:
        if len(i) == 1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)      

In [ ]:
sub = []
rest_image = []
for i in test:
    sub.append(i[0])
for image in test_images:
    if image not in sub:
        rest_image.append(image) 


detector=MTCNN()
test_df_ = []
for image in rest_image:
    img = cv2.imread(os.path.join(images_dir,image))
    faces = detector.detect_faces(img)
    test_ = []
    for face in faces:
        bounding_box = face['box']
        test_.append([image, bounding_box])
    test_df_.append(test_) 
for i in test_df_:
    if len(i) > 0:
        if len(i) == 1:
            test.append(i[0])
        else:
            for j in i:
                test.append(j)      

In [ ]:
negative = []
for i in test:
    for j in i[1]:
        if j < 0:
            negative.append(i)
    

In [ ]:
test_data=[]
def create_test_data():
            for j in test:
                if j not in negative:
                    img = cv2.imread(os.path.join(images_dir, j[0]), cv2.IMREAD_GRAYSCALE)
                    img = img[j[1][1]:j[1][1]+j[1][3],j[1][0]:j[1][0]+j[1][2]]
                    new_img = cv2.resize(img,(50,50))
                    new_img = new_img.reshape(-1,50,50,1)
                    predict = model.predict(new_img)
                    test_data.append([j, predict])

create_test_data()      

In [ ]:
image = []
classname = []
for i, j in test_data:
    classname.append(np.argmax(j))
    image.append(i)
df=pd.DataFrame(columns=['image', 'classname'])
df['image'] = image
df['classname'] = classname
df['classname'] = lbl.inverse_transform(df['classname'])
image = []
x1 = []
x2 = []
y1 = []
y2 = []
for i in df['image']:
    image.append(i[0])
    x1.append(i[1][0])
    x2.append(i[1][1])
    y1.append(i[1][2])
    y2.append(i[1][3])
df['name'] = image
df['x1'] = x1
df['x2'] = x2
df['y1'] = y1
df['y2'] = y2    
df.drop(['image'],axis=1,inplace=True)

In [ ]:
df.sort_values('name', axis=0, inplace=True, ascending=False)
# df.to_csv('submission_1.csv')
df

In [ ]:
df.sort_values('name', axis=0, inplace=True, ascending=False)
# df.to_csv('submission_1.csv')
df